# Fair RS

## Questions and Notes

### Questions
1. How do we distribute documents?
2. How does reading a document influence a user?
3. How do we simulate a user's past/initial leaning?
4. What set of users do we start from? Can we base it on real life or a dataset?
5. What features does each document have (clickbaityness, topic, political leaning, quality, ...)?
6. How do we define engagement?
7. How can we drive a user into a certain direction?
8. What assumptions are legitimate? How do you make good assumptions in modeling?

### Notes
**Initial Assumptions:**
- If a piece is aligned with a person's interest, they prefer it
- We present 5 pieces of content to each user 
- When presented with content, the user clicks on 0 or 1 piece of content^
- Content-user response is dependent on | leaning - affinity | is small, the user-response
- Every slate (i.e. content recommendation) is treated as a single action (don't consider past )
- Content has attributes:
    - leaning
    - clickbaityness
- User has attributes:
    - affinity/opinion
    - satisfaction
- User response has engagement as main characteristic

-> Borrow ideas from both interest_explortion and long_term_satisfaction

**Advanced Assumptions:**
- On each topic, a user has an opinion and a piece of content also has a leaning. The closer opinion and leaning are, the more satisfaction a user derives from the piece of content
- If a user likes a piece content their opinion shifts in the direction of the content's leaning  
- Users generally do not notice/dislike that they are in a filter bubble
- While content might have a general leaning, liking/disliking is more complex than that -> I might agree with the infrastructure bill even though I am a republican -> This effect is stronger the closer to the edge of my filter bubble I am
- We could research some social science studies on filter bubbles and why people click on what to model it

## Driver Code

### Environment

In [1]:
#@title Importing generics
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [2]:
#@title Importing RecSim components 
from recsim.agents import full_slate_q_agent
from recsim.environments import long_term_satisfaction
from recsim.simulator import runner_lib

In [3]:
seed = 0
np.random.seed(seed)

### Code

In [4]:
def create_agent(sess, environment, eval_mode, summary_writer=None):
  kwargs = {
      'observation_space': environment.observation_space,
      'action_space': environment.action_space,
      'summary_writer': summary_writer,
      'eval_mode': eval_mode,
  }
  return full_slate_q_agent.FullSlateQAgent(sess, **kwargs)

In [5]:
# - slate_size sets the size of the set of elements presented to the user;
# - num_candidates specifies the number of documents present in the document database at any given time;
# - resample_documents specifies whether the set of candidates should be resampled between time steps according to the document distribution (more on this in [later notebooks](RecSim_Developing_an_Environment.ipynb)).
#   finally, we set the random seed.

env_config = {
  'num_candidates': 20,
  'slate_size': 5,
  'resample_documents': True,
  'seed': seed,
  }

In [10]:
#@title Training the agent
tmp_base_dir = '/tmp/recsim/'
runner = runner_lib.TrainRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=long_term_satisfaction.create_environment(env_config),
    episode_log_file="",
    max_steps_per_episode=5,
    max_training_steps=1,
    num_iterations=1)
runner.run_experiment()

INFO:tensorflow:max_training_steps = 1, number_iterations = 1,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_training_steps = 1, number_iterations = 1,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_steps_per_episode = 5


INFO:tensorflow:max_steps_per_episode = 5


In [ ]:
#@title Evaluating the agent
runner = runner_lib.EvalRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=long_term_satisfaction.create_environment(env_config),
    max_eval_episodes=5,
    test_mode=True)
runner.run_experiment()

In [ ]:
#@title Tensorboard
%tensorboard --logdir=/tmp/recsim/